In [2]:
# Written by Bob Zhang. Designed to facilitate UCSD HDH's calendar import from events organized in excel table. 

import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from datetime import datetime
from icalendar import Calendar, Event

# open the excel and parse all the columns into lists
df = pd.read_excel('ERCDeskEvents.xls', sheet_name='Sheet1') # df for data frame
listConfName = df['Conference Name']
listHousArea = df['College Housing Area']
listConfCord = df['Conference Coordinator']
listArrvDate = df['Arrival 1 : Date']
listArrTm_Ct = df['Arrival 1 : Time, Count']
listChkInstr = df['Arrival 1 : Check-In Instructions']
listMobi_Plc = df['Mobile Check-In Location  & Date']

# split arrival time & people count
for i in range(0, 150): 
    listArrTm_Ct[i] = listArrTm_Ct[i].split(', ')

cal = Calendar()

# iterate through the table to create .ics file, 150 is the last for ERC desk in this file
for i in range(5, 6):
    print('conference name: ', listConfName[i])
    event = Event()
        
    # date start & end
    peopleCount = listArrTm_Ct[i][1]
    
    if '-' in listArrTm_Ct[i][0]:
        separator = listArrTm_Ct[i][0].index('-')
    else:
        separator = listArrTm_Ct[i][0].index(':')
    beginTime = listArrTm_Ct[i][0][0:separator].split(':')
    
    
    try: 
        separator = listArrTm_Ct[i][0].index('-') # has a end time
        beginTime = listArrTm_Ct[i][0][0:separator].split(':')
        print(beginTime)
        endTime = listArrTm_Ct[i][0][separator + 1:len(listArrTm_Ct[i][0]) + 1].split(':')
    except ValueError: # no end time b/c no '-' in arrival time: assume 1 hour long 
        begin = listArrTm_Ct[i][0].index('m')
        beginTime = listArrTm_Ct[i][0][0:(begin + 1)]
        endHour = int(beginTime[0:beginTime.index(':')]) + 1
        endTime = str(endHour) + ':' + beginTime[beginTime.index(':') + 1:len(beginTime) + 1]
        
        
        
        
    print(beginTime)
    begin = datetime(int(listArrvDate[i].year), int(listArrvDate[i].month), int(listArrvDate[i].day), 
                     int(beginTime[0]), int(beginTime[1][0:2]))
    end = datetime(int(listArrvDate[i].year), int(listArrvDate[i].month), int(listArrvDate[i].day), 
                   int(endTime[0]), int(endTime[1][0:2]))
    
    # do something else
    summary = listConfName[i] + ' (' + str(peopleCount) + ')'
    
    # categorize
    if listMobi_Plc[i] == "nan": 
        # event.add('CATEGORIES', 'Check-in')
        summary = summary + '(Check-in)'
    # elif (font is red): 
    #     event.add('categories, 'Check-out')
    else: 
        # event.add('CATEGORIES', 'Blue Category')
        summary += '(Mobile Check-in)'
    
    event.add('summary', summary)
    event.add('description', listChkInstr[i])
    event.add('dtstart', begin)
    event.add('dtend', end)
    cal.add_component(event)

    filename = summary + '.ics'
    print('filename', filename)
    file = open(filename, 'wb')
    file.write(cal.to_ical())
    file.close()


conference name:  Alexa Café by iD Tech Camps-Week 2
6:00pm


ValueError: invalid literal for int() with base 10: ':'